In [16]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys 
sys.path.append(os.path.abspath("../../src"))   
import soporte_preprocesamiento as f
import warnings
warnings.filterwarnings("ignore")


from datetime import datetime

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
pd.set_option("display.max_columns", None)
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler

In [17]:
df_null_encoded=pd.read_csv("../../datos/datos4/datos_nulos_encoded.csv", index_col=0).reset_index(drop=True)

In [18]:
df_null_encoded.columns

Index(['price', 'size', 'rooms', 'municipality', 'distance', 'status', 'floor',
       'hasLift', 'propertyType_chalet', 'propertyType_countryHouse',
       ...
       'district_Vicálvaro', 'district_Villa de Vallecas',
       'district_Villalba Estación', 'district_Villaverde',
       'district_Villayuventus-Renfe', 'district_Zona Estación',
       'district_Zona Estación- Centro', 'district_Zona Industrial',
       'district_Zona Pueblo', 'district_desconocido'],
      dtype='object', length=102)

In [19]:
df_null_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Columns: 102 entries, price to district_desconocido
dtypes: float64(101), int64(1)
memory usage: 277.4 KB


In [20]:
df_null_encoded.head()

,price,size,rooms,municipality,distance,status,floor,hasLift,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,exterior_False,exterior_True,bathrooms_1,bathrooms_2,bathrooms_3,district_Alcobendas Centro,district_Arganzuela,district_Barajas,district_Barrio de Salamanca,district_Buenavista,district_Carabanchel,district_Carlos Ruiz,district_Casco Antiguo,district_Casco Histórico,district_Casco Urbano,district_Centro,district_Centro - Casco Histórico,district_Centro Urbano,district_Chamartín,district_Chamberí,district_Chorrillo,district_Ciudad 70,district_Ciudad Lineal,district_Dehesa - El Soto,district_El Mirador,district_El Mirador - Grillero,district_El Nido-Las Fuentes,district_Ensanche,district_Espartales,district_Foso-Moreras,district_Fuencarral,district_Fuentebella-San Felix-El Leguario,district_Getafe Centro,district_Getafe norte,district_Hortaleza,district_Hospital,district_Juan de Austria,district_La Alhóndiga,district_La Dehesa - El Pinar,district_La Espinilla - Parque Blanco,district_Las Américas,district_Las Matas- Peñascales,district_Las Sedas - El Olivar,district_Latina,district_Los Llanos - Valle Pardo,district_Moncloa,district_Montserrat - Parque Empresarial,district_Noroeste,district_Nuevo Aranjuez-Ciudad de las Artes,district_Parla Este,district_Parque Europa - Los Pitufos,district_Parque Inlasa,district_Parque Roma - Coronas,district_Parque de la Coruña - Las Suertes,district_Pintores-Ferial,district_Pol. Industrial sur,district_Pryconsa - Poligono Europa,district_Puente de Vallecas,district_Retiro,district_Reyes,district_Reyes Católicos,district_Rivas Futura,district_San Blas,district_San Crispín - La Estación Consorcio,district_San Isidro,district_San Isidro - Los Almendros,district_San José - Buenos Aires,district_Sudeste Industrial,district_Suroeste,district_Tetuán,district_Universidad,district_Usera,district_Val,district_Valdemorillo pueblo,district_Valdepelayo - Montepinos - Arroyo Culebro,district_Valderas - Los Castillos,district_Valleaguado - La Cañada,district_Vega de la Moraleja,district_Vicálvaro,district_Villa de Vallecas,district_Villalba Estación,district_Villaverde,district_Villayuventus-Renfe,district_Zona Estación,district_Zona Estación- Centro,district_Zona Industrial,district_Zona Pueblo,district_desconocido
0,750.0,60.0,682.653334,705.594114,7037,695.684375,697.783501,702.497108,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,750.0,70.0,697.968657,701.798146,16145,695.684375,678.674367,687.368755,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,400.0,67.0,697.968657,655.230673,55041,695.684375,696.104807,687.368755,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,590.0,70.0,708.592894,679.951283,56596,695.684375,672.172228,687.368755,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [21]:


# Definir las variables y escaladores
variables =[
    "price", "size", "rooms", "municipality", "distance", "status", "floor", "hasLift",
    "propertyType_chalet", "propertyType_countryHouse", "propertyType_duplex", "propertyType_flat",
    "propertyType_penthouse", "propertyType_studio", "exterior_False", "exterior_True", "bathrooms_1",
    "bathrooms_2", "bathrooms_3", "district_Alcobendas Centro", "district_Arganzuela", "district_Barajas",
    "district_Barrio de Salamanca", "district_Buenavista", "district_Carabanchel", "district_Carlos Ruiz",
    "district_Casco Antiguo", "district_Casco Histórico", "district_Casco Urbano", "district_Centro",
    "district_Centro - Casco Histórico", "district_Centro Urbano", "district_Chamartín", "district_Chamberí",
    "district_Chorrillo", "district_Ciudad 70", "district_Ciudad Lineal", "district_Dehesa - El Soto",
    "district_El Mirador", "district_El Mirador - Grillero", "district_El Nido-Las Fuentes", "district_Ensanche",
    "district_Espartales", "district_Foso-Moreras", "district_Fuencarral", "district_Fuentebella-San Felix-El Leguario",
    "district_Getafe Centro", "district_Getafe norte", "district_Hortaleza", "district_Hospital",
    "district_Juan de Austria", "district_La Alhóndiga", "district_La Dehesa - El Pinar",
    "district_La Espinilla - Parque Blanco", "district_Las Américas", "district_Las Matas- Peñascales",
    "district_Las Sedas - El Olivar", "district_Latina", "district_Los Llanos - Valle Pardo", "district_Moncloa",
    "district_Montserrat - Parque Empresarial", "district_Noroeste", "district_Nuevo Aranjuez-Ciudad de las Artes",
    "district_Parla Este", "district_Parque Europa - Los Pitufos", "district_Parque Inlasa",
    "district_Parque Roma - Coronas", "district_Parque de la Coruña - Las Suertes", "district_Pintores-Ferial",
    "district_Pol. Industrial sur", "district_Pryconsa - Poligono Europa", "district_Puente de Vallecas",
    "district_Retiro", "district_Reyes", "district_Reyes Católicos", "district_Rivas Futura", "district_San Blas",
    "district_San Crispín - La Estación Consorcio", "district_San Isidro", "district_San Isidro - Los Almendros",
    "district_San José - Buenos Aires", "district_Sudeste Industrial", "district_Suroeste", "district_Tetuán",
    "district_Universidad", "district_Usera", "district_Val", "district_Valdemorillo pueblo",
    "district_Valdepelayo - Montepinos - Arroyo Culebro", "district_Valderas - Los Castillos",
    "district_Valleaguado - La Cañada", "district_Vega de la Moraleja", "district_Vicálvaro",
    "district_Villa de Vallecas", "district_Villalba Estación", "district_Villaverde",
    "district_Villayuventus-Renfe", "district_Zona Estación", "district_Zona Estación- Centro",
    "district_Zona Industrial", "district_Zona Pueblo", "district_desconocido"
]


escaladores = {
    "robust": RobustScaler(),
    "MinMax": MinMaxScaler(),
    "norm": Normalizer(),
    "est": StandardScaler()
}

# Crear nuevas columnas escaladas
for esc_name, escalador in escaladores.items():
    for var in variables:
        # Transformar las variables con el escalador
        datos_transformados = escalador.fit_transform(df_null_encoded[[var]])
        # Crear una nueva columna en el DataFrame con el nombre del escalador
        df_null_encoded[f"{var}_{esc_name}"] = datos_transformados

# Visualizar la descripción del DataFrame resultante
df_null_encoded.describe().T.head()


,count,mean,std,min,25%,50%,75%,max
price,348.0,693.405172,61.418015,400.000000,656.000000,707.500000,745.500000,750.000000
size,348.0,58.086207,19.469042,20.000000,45.000000,56.000000,70.000000,133.000000
rooms,348.0,693.164866,9.027034,682.653334,682.653334,697.968657,697.968657,708.592894
municipality,348.0,698.951978,8.986833,655.230673,696.193356,701.798146,705.594114,705.594114
distance,348.0,16348.988506,13320.680886,183.000000,5615.250000,11972.500000,25183.750000,56596.000000


In [22]:
df_null_encoded.head(2)

,price,size,rooms,municipality,distance,status,floor,hasLift,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,exterior_False,exterior_True,bathrooms_1,bathrooms_2,bathrooms_3,district_Alcobendas Centro,district_Arganzuela,district_Barajas,district_Barrio de Salamanca,district_Buenavista,district_Carabanchel,district_Carlos Ruiz,district_Casco Antiguo,district_Casco Histórico,district_Casco Urbano,district_Centro,district_Centro - Casco Histórico,district_Centro Urbano,district_Chamartín,district_Chamberí,district_Chorrillo,district_Ciudad 70,district_Ciudad Lineal,district_Dehesa - El Soto,district_El Mirador,district_El Mirador - Grillero,district_El Nido-Las Fuentes,district_Ensanche,district_Espartales,district_Foso-Moreras,district_Fuencarral,district_Fuentebella-San Felix-El Leguario,district_Getafe Centro,district_Getafe norte,district_Hortaleza,district_Hospital,district_Juan de Austria,district_La Alhóndiga,district_La Dehesa - El Pinar,district_La Espinilla - Parque Blanco,district_Las Américas,district_Las Matas- Peñascales,district_Las Sedas - El Olivar,district_Latina,district_Los Llanos - Valle Pardo,district_Moncloa,district_Montserrat - Parque Empresarial,district_Noroeste,district_Nuevo Aranjuez-Ciudad de las Artes,district_Parla Este,district_Parque Europa - Los Pitufos,district_Parque Inlasa,district_Parque Roma - Coronas,district_Parque de la Coruña - Las Suertes,district_Pintores-Ferial,district_Pol. Industrial sur,district_Pryconsa - Poligono Europa,district_Puente de Vallecas,district_Retiro,district_Reyes,district_Reyes Católicos,district_Rivas Futura,district_San Blas,district_San Crispín - La Estación Consorcio,district_San Isidro,district_San Isidro - Los Almendros,district_San José - Buenos Aires,district_Sudeste Industrial,district_Suroeste,district_Tetuán,district_Universidad,district_Usera,district_Val,district_Valdemorillo pueblo,district_Valdepelayo - Montepinos - Arroyo Culebro,district_Valderas - Los Castillos,district_Valleaguado - La Cañada,district_Vega de la Moraleja,district_Vicálvaro,district_Villa de Vallecas,district_Villalba Estación,district_Villaverde,district_Villayuventus-Renfe,district_Zona Estación,district_Zona Estación- Centro,district_Zona Industrial,district_Zona Pueblo,district_desconocido,price_robust,size_robust,rooms_robust,municipality_robust,distance_robust,status_robust,floor_robust,hasLift_robust,propertyType_chalet_robust,propertyType_countryHouse_robust,propertyType_duplex_robust,propertyType_flat_robust,propertyType_penthouse_robust,propertyType_studio_robust,exterior_False_robust,exterior_True_robust,bathrooms_1_robust,bathrooms_2_robust,bathrooms_3_robust,district_Alcobendas Centro_robust,district_Arganzuela_robust,district_Barajas_robust,district_Barrio de Salamanca_robust,district_Buenavista_robust,district_Carabanchel_robust,district_Carlos Ruiz_robust,district_Casco Antiguo_robust,district_Casco Histórico_robust,district_Casco Urbano_robust,district_Centro_robust,district_Centro - Casco Histórico_robust,district_Centro Urbano_robust,district_Chamartín_robust,district_Chamberí_robust,district_Chorrillo_robust,district_Ciudad 70_robust,district_Ciudad Lineal_robust,district_Dehesa - El Soto_robust,district_El Mirador_robust,district_El Mirador - Grillero_robust,district_El Nido-Las Fuentes_robust,district_Ensanche_robust,district_Espartales_robust,district_Foso-Moreras_robust,district_Fuencarral_robust,district_Fuentebella-San Felix-El Leguario_robust,district_Getafe Centro_robust,district_Getafe norte_robust,district_Hortaleza_robust,district_Hospital_robust,district_Juan de Austria_robust,district_La Alhóndiga_robust,district_La Dehesa - El Pinar_robust,district_La Espinilla - Parque Blanco_robust,district_Las Américas_robust,district_Las Matas- Peñascales_robust,district_Las Sedas - El Olivar_robust,district_Latina_robust,district_Los Llanos - Valle Pardo_robust,district_Moncloa_r

In [23]:
df_null_encoded.columns


Index(['price', 'size', 'rooms', 'municipality', 'distance', 'status', 'floor',
       'hasLift', 'propertyType_chalet', 'propertyType_countryHouse',
       ...
       'district_Vicálvaro_est', 'district_Villa de Vallecas_est',
       'district_Villalba Estación_est', 'district_Villaverde_est',
       'district_Villayuventus-Renfe_est', 'district_Zona Estación_est',
       'district_Zona Estación- Centro_est', 'district_Zona Industrial_est',
       'district_Zona Pueblo_est', 'district_desconocido_est'],
      dtype='object', length=510)

In [24]:
# df_num=df_sin_null.select_dtypes(np.number)
# df_num.drop(columns="price", inplace=True)
# lista_cols=df_num.columns
# lista_cols_num=lista_cols.to_list()

In [25]:
# df_robust, scaler=f.escalador_datos(df_sin_null, lista_cols_num, method = "robust")     #el scaler es para el streamlit futuro

Dado que en el proceso de limpieza y exploración se observaron y se seleccionaron los límites de lo que se consideraría outlier y lo que se considera datos erróneos, los outliers que nos aparecen aquí son datos reales con lo que si que me interesa mantenerlos.  
Es por ello que tras descartar el método de normalizar y el de escalado min y max. Tuve que elegir entre la metodología robusta y la estándar.  
Finalmente, me decanto por la robusta ya que le da más importancia a dichos outliers que ya hemos comprobado que son datos viables y coherentes.

*Entrando un poco más en detalle:*  
Los kilómetros no contaban con una cantidad ingente de outliers a diferencia de la potencia que contaba con una proporción muy alta.  
Es por ello que al comparar las distribuciones tanto entre métodos como con la variable original, tanto el robusto como el estandarizado se parecen tanto.  
  
Además, tienen sentido las distribuciones ya que al ser una tienda de coches usados, los kilómetros tienen sentido que se agrupen en los valores más altos al igual que tiene sentido que la potencia se agrupe en valores más bajos ya que no son habituales los coches con una potencia superior a 400.

In [26]:
df_null_encoded.columns

Index(['price', 'size', 'rooms', 'municipality', 'distance', 'status', 'floor',
       'hasLift', 'propertyType_chalet', 'propertyType_countryHouse',
       ...
       'district_Vicálvaro_est', 'district_Villa de Vallecas_est',
       'district_Villalba Estación_est', 'district_Villaverde_est',
       'district_Villayuventus-Renfe_est', 'district_Zona Estación_est',
       'district_Zona Estación- Centro_est', 'district_Zona Industrial_est',
       'district_Zona Pueblo_est', 'district_desconocido_est'],
      dtype='object', length=510)

In [ ]:
lista_eliminar=[
    "price_est", "size_est", "rooms_est", "municipality_est", "distance_est", "status_est", "floor_est", "hasLift_est",
    "propertyType_chalet_est", "propertyType_countryHouse_est", "propertyType_duplex_est", "propertyType_flat_est",
    "propertyType_penthouse_est", "propertyType_studio_est", "exterior_False_est", "exterior_True_est",
    "bathrooms_1_est", "bathrooms_2_est", "bathrooms_3_est", "district_Alcobendas Centro_est",
    "district_Arganzuela_est", "district_Barajas_est", "district_Barrio de Salamanca_est", "district_Buenavista_est",
    "district_Carabanchel_est", "district_Carlos Ruiz_est", "district_Casco Antiguo_est", "district_Casco Histórico_est",
    "district_Casco Urbano_est", "district_Centro_est", "district_Centro - Casco Histórico_est", "district_Centro Urbano_est",
    "district_Chamartín_est", "district_Chamberí_est", "district_Chorrillo_est", "district_Ciudad 70_est",
    "district_Ciudad Lineal_est", "district_Dehesa - El Soto_est", "district_El Mirador_est", "district_El Mirador - Grillero_est",
    "district_El Nido-Las Fuentes_est", "district_Ensanche_est", "district_Espartales_est", "district_Foso-Moreras_est",
    "district_Fuencarral_est", "district_Fuentebella-San Felix-El Leguario_est", "district_Getafe Centro_est",
    "district_Getafe norte_est", "district_Hortaleza_est", "district_Hospital_est", "district_Juan de Austria_est",
    "district_La Alhóndiga_est", "district_La Dehesa - El Pinar_est", "district_La Espinilla - Parque Blanco_est",
    "district_Las Américas_est", "district_Las Matas- Peñascales_est", "district_Las Sedas - El Olivar_est",
    "district_Latina_est", "district_Los Llanos - Valle Pardo_est", "district_Moncloa_est", "district_Montserrat - Parque Empresarial_est",
    "district_Noroeste_est", "district_Nuevo Aranjuez-Ciudad de las Artes_est", "district_Parla Este_est",
    "district_Parque Europa - Los Pitufos_est", "district_Parque Inlasa_est", "district_Parque Roma - Coronas_est",
    "district_Parque de la Coruña - Las Suertes_est", "district_Pintores-Ferial_est", "district_Pol. Industrial sur_est",
    "district_Pryconsa - Poligono Europa_est", "district_Puente de Vallecas_est", "district_Retiro_est", "district_Reyes_est",
    "district_Reyes Católicos_est", "district_Rivas Futura_est", "district_San Blas_est", "district_San Crispín - La Estación Consorcio_est",
    "district_San Isidro_est", "district_San Isidro - Los Almendros_est", "district_San José - Buenos Aires_est",
    "district_Sudeste Industrial_est", "district_Suroeste_est", "district_Tetuán_est", "district_Universidad_est",
    "district_Usera_est", "district_Val_est", "district_Valdemorillo pueblo_est", "district_Valdepelayo - Montepinos - Arroyo Culebro_est",
    "district_Valderas - Los Castillos_est", "district_Valleaguado - La Cañada_est", "district_Vega de la Moraleja_est",
    "district_Vicálvaro_est", "district_Villa de Vallecas_est", "district_Villalba Estación_est", "district_Villaverde_est",
    "district_Villayuventus-Renfe_est", "district_Zona Estación_est", "district_Zona Estación- Centro_est",
    "district_Zona Industrial_est", "district_Zona Pueblo_est", "district_desconocido_est",
    "price_norm", "size_norm", "rooms_norm", "municipality_norm", "distance_norm", "status_norm", "floor_norm", "hasLift_norm",
    "propertyType_chalet_norm", "propertyType_countryHouse_norm", "propertyType_duplex_norm", "propertyType_flat_norm",
    "propertyType_penthouse_norm", "propertyType_studio_norm", "exterior_False_norm", "exterior_True_norm",
    "bathrooms_1_norm", "bathrooms_2_norm", "bathrooms_3_norm", "district_Alcobendas Centro_norm",
    "district_Arganzuela_norm", "district_Barajas_norm", "district_Barrio de Salamanca_norm", "district_Buenavista_norm",
    "district_Carabanchel_norm", "district_Carlos Ruiz_norm", "district_Casco Antiguo_norm", "district_Casco Histórico_norm",
    "district_Casco Urbano_norm", "district_Centro_norm", "district_Centro - Casco Histórico_norm", "district_Centro Urbano_norm",
    "district_Chamartín_norm", "district_Chamberí_norm", "district_Chorrillo_norm", "district_Ciudad 70_norm",
    "district_Ciudad Lineal_norm", "district_Dehesa - El Soto_norm", "district_El Mirador_norm", "district_El Mirador - Grillero_norm",
    "district_El Nido-Las Fuentes_norm", "district_Ensanche_norm", "district_Espartales_norm", "district_Foso-Moreras_norm",
    "district_Fuencarral_norm", "district_Fuentebella-San Felix-El Leguario_norm", "district_Getafe Centro_norm",
    "district_Getafe norte_norm", "district_Hortaleza_norm", "district_Hospital_norm", "district_Juan de Austria_norm",
    "district_La Alhóndiga_norm", "district_La Dehesa - El Pinar_norm", "district_La Espinilla - Parque Blanco_norm",
    "district_Las Américas_norm", "district_Las Matas- Peñascales_norm", "district_Las Sedas - El Olivar_norm",
    "district_Latina_norm", "district_Los Llanos - Valle Pardo_norm", "district_Moncloa_norm", "district_Montserrat - Parque Empresarial_norm",
    "district_Noroeste_norm", "district_Nuevo Aranjuez-Ciudad de las Artes_norm", "district_Parla Este_norm",
    "district_Parque Europa - Los Pitufos_norm", "district_Parque Inlasa_norm", "district_Parque Roma - Coronas_norm",
    "district_Parque de la Coruña - Las Suertes_norm", "district_Pintores-Ferial_norm", "district_Pol. Industrial sur_norm",
    "district_Pryconsa - Poligono Europa_norm", "district_Puente de Vallecas_norm", "district_Retiro_norm", "district_Reyes_norm",
    "district_Reyes Católicos_norm", "district_Rivas Futura_norm", "district_San Blas_norm", "district_San Crispín - La Estación Consorcio_norm",
    "district_San Isidro_norm", "district_San Isidro - Los Almendros_norm", "district_San José - Buenos Aires_norm",
    "district_Sudeste Industrial_norm", "district_Suroeste_norm", "district_Tetuán_norm", "district_Universidad_norm",
    "district_Usera_norm", "district_Val_norm", "district_Valdemorillo pueblo_norm", "district_Valdepelayo - Montepinos - Arroyo Culebro_norm",
    "district_Valderas - Los Castillos_norm", "district_Valleaguado - La Cañada_norm", "district_Vega de la Moraleja_norm",
    "district_Vicálvaro_norm", "district_Villa de Vallecas_norm", "district_Villalba Estación_norm", "district_Villaverde_norm",
    "district_Villayuventus-Renfe_norm", "district_Zona Estación_norm", "district_Zona Estación- Centro_norm",
    "district_Zona Industrial_norm", "district_Zona Pueblo_norm", "district_desconocido_norm",
    "price_MinMax","price_robust", "size_robust", "rooms_robust", "municipality_robust", "distance_robust", "status_robust", "floor_robust", "hasLift_robust",
    "propertyType_chalet_robust", "propertyType_countryHouse_robust", "propertyType_duplex_robust", "propertyType_flat_robust",
    "propertyType_penthouse_robust", "propertyType_studio_robust", "exterior_False_robust", "exterior_True_robust",
    "bathrooms_1_robust", "bathrooms_2_robust", "bathrooms_3_robust", "district_Alcobendas Centro_robust",
    "district_Arganzuela_robust", "district_Barajas_robust", "district_Barrio de Salamanca_robust", "district_Buenavista_robust",
    "district_Carabanchel_robust", "district_Carlos Ruiz_robust", "district_Casco Antiguo_robust", "district_Casco Histórico_robust",
    "district_Casco Urbano_robust", "district_Centro_robust", "district_Centro - Casco Histórico_robust", "district_Centro Urbano_robust",
    "district_Chamartín_robust", "district_Chamberí_robust", "district_Chorrillo_robust", "district_Ciudad 70_robust",
    "district_Ciudad Lineal_robust", "district_Dehesa - El Soto_robust", "district_El Mirador_robust", "district_El Mirador - Grillero_robust",
    "district_El Nido-Las Fuentes_robust", "district_Ensanche_robust", "district_Espartales_robust", "district_Foso-Moreras_robust",
    "district_Fuencarral_robust", "district_Fuentebella-San Felix-El Leguario_robust", "district_Getafe Centro_robust",
    "district_Getafe norte_robust", "district_Hortaleza_robust", "district_Hospital_robust", "district_Juan de Austria_robust",
    "district_La Alhóndiga_robust", "district_La Dehesa - El Pinar_robust", "district_La Espinilla - Parque Blanco_robust",
    "district_Las Américas_robust", "district_Las Matas- Peñascales_robust", "district_Las Sedas - El Olivar_robust",
    "district_Latina_robust", "district_Los Llanos - Valle Pardo_robust", "district_Moncloa_robust", "district_Montserrat - Parque Empresarial_robust",
    "district_Noroeste_robust", "district_Nuevo Aranjuez-Ciudad de las Artes_robust", "district_Parla Este_robust",
    "district_Parque Europa - Los Pitufos_robust", "district_Parque Inlasa_robust", "district_Parque Roma - Coronas_robust",
    "district_Parque de la Coruña - Las Suertes_robust", "district_Pintores-Ferial_robust", "district_Pol. Industrial sur_robust",
    "district_Pryconsa - Poligono Europa_robust", "district_Puente de Vallecas_robust", "district_Retiro_robust", "district_Reyes_robust",
    "district_Reyes Católicos_robust", "district_Rivas Futura_robust", "district_San Blas_robust", "district_San Crispín - La Estación Consorcio_robust",
    "district_San Isidro_robust", "district_San Isidro - Los Almendros_robust", "district_San José - Buenos Aires_robust",
    "district_Sudeste Industrial_robust", "district_Suroeste_robust", "district_Tetuán_robust", "district_Universidad_robust",
    "district_Usera_robust", "district_Val_robust", "district_Valdemorillo pueblo_robust", "district_Valdepelayo - Montepinos - Arroyo Culebro_robust",
    "district_Valderas - Los Castillos_robust", "district_Valleaguado - La Cañada_robust", "district_Vega de la Moraleja_robust",
    "district_Vicálvaro_robust", "district_Villa de Vallecas_robust", "district_Villalba Estación_robust", "district_Villaverde_robust",
    "district_Villayuventus-Renfe_robust", "district_Zona Estación_robust", "district_Zona Estación- Centro_robust",
    "district_Zona Industrial_robust", "district_Zona Pueblo_robust", "district_desconocido_robust",
    "size", "rooms", "municipality", "distance", "status", "floor", "hasLift",
    "propertyType_chalet", "propertyType_countryHouse", "propertyType_duplex", "propertyType_flat",
    "propertyType_penthouse", "propertyType_studio", "exterior_False", "exterior_True", "bathrooms_1",
    "bathrooms_2", "bathrooms_3", "district_Alcobendas Centro", "district_Arganzuela", "district_Barajas",
    "district_Barrio de Salamanca", "district_Buenavista", "district_Carabanchel", "district_Carlos Ruiz",
    "district_Casco Antiguo", "district_Casco Histórico", "district_Casco Urbano", "district_Centro",
    "district_Centro - Casco Histórico", "district_Centro Urbano", "district_Chamartín", "district_Chamberí",
    "district_Chorrillo", "district_Ciudad 70", "district_Ciudad Lineal", "district_Dehesa - El Soto",
    "district_El Mirador", "district_El Mirador - Grillero", "district_El Nido-Las Fuentes", "district_Ensanche",
    "district_Espartales", "district_Foso-Moreras", "district_Fuencarral", "district_Fuentebella-San Felix-El Leguario",
    "district_Getafe Centro", "district_Getafe norte", "district_Hortaleza", "district_Hospital",
    "district_Juan de Austria", "district_La Alhóndiga", "district_La Dehesa - El Pinar",
    "district_La Espinilla - Parque Blanco", "district_Las Américas", "district_Las Matas- Peñascales",
    "district_Las Sedas - El Olivar", "district_Latina", "district_Los Llanos - Valle Pardo", "district_Moncloa",
    "district_Montserrat - Parque Empresarial", "district_Noroeste", "district_Nuevo Aranjuez-Ciudad de las Artes",
    "district_Parla Este", "district_Parque Europa - Los Pitufos", "district_Parque Inlasa",
    "district_Parque Roma - Coronas", "district_Parque de la Coruña - Las Suertes", "district_Pintores-Ferial",
    "district_Pol. Industrial sur", "district_Pryconsa - Poligono Europa", "district_Puente de Vallecas",
    "district_Retiro", "district_Reyes", "district_Reyes Católicos", "district_Rivas Futura", "district_San Blas",
    "district_San Crispín - La Estación Consorcio", "district_San Isidro", "district_San Isidro - Los Almendros",
    "district_San José - Buenos Aires", "district_Sudeste Industrial", "district_Suroeste", "district_Tetuán",
    "district_Universidad", "district_Usera", "district_Val", "district_Valdemorillo pueblo",
    "district_Valdepelayo - Montepinos - Arroyo Culebro", "district_Valderas - Los Castillos",
    "district_Valleaguado - La Cañada", "district_Vega de la Moraleja", "district_Vicálvaro",
    "district_Villa de Vallecas", "district_Villalba Estación", "district_Villaverde",
    "district_Villayuventus-Renfe", "district_Zona Estación", "district_Zona Estación- Centro",
    "district_Zona Industrial", "district_Zona Pueblo", "district_desconocido"
]


#también las originales

In [28]:
df_null_encoded.drop(columns=lista_eliminar, inplace=True)

In [29]:
df_null_encoded.head(2)

,price,size_MinMax,rooms_MinMax,municipality_MinMax,distance_MinMax,status_MinMax,floor_MinMax,hasLift_MinMax,propertyType_chalet_MinMax,propertyType_countryHouse_MinMax,propertyType_duplex_MinMax,propertyType_flat_MinMax,propertyType_penthouse_MinMax,propertyType_studio_MinMax,exterior_False_MinMax,exterior_True_MinMax,bathrooms_1_MinMax,bathrooms_2_MinMax,bathrooms_3_MinMax,district_Alcobendas Centro_MinMax,district_Arganzuela_MinMax,district_Barajas_MinMax,district_Barrio de Salamanca_MinMax,district_Buenavista_MinMax,district_Carabanchel_MinMax,district_Carlos Ruiz_MinMax,district_Casco Antiguo_MinMax,district_Casco Histórico_MinMax,district_Casco Urbano_MinMax,district_Centro_MinMax,district_Centro - Casco Histórico_MinMax,district_Centro Urbano_MinMax,district_Chamartín_MinMax,district_Chamberí_MinMax,district_Chorrillo_MinMax,district_Ciudad 70_MinMax,district_Ciudad Lineal_MinMax,district_Dehesa - El Soto_MinMax,district_El Mirador_MinMax,district_El Mirador - Grillero_MinMax,district_El Nido-Las Fuentes_MinMax,district_Ensanche_MinMax,district_Espartales_MinMax,district_Foso-Moreras_MinMax,district_Fuencarral_MinMax,district_Fuentebella-San Felix-El Leguario_MinMax,district_Getafe Centro_MinMax,district_Getafe norte_MinMax,district_Hortaleza_MinMax,district_Hospital_MinMax,district_Juan de Austria_MinMax,district_La Alhóndiga_MinMax,district_La Dehesa - El Pinar_MinMax,district_La Espinilla - Parque Blanco_MinMax,district_Las Américas_MinMax,district_Las Matas- Peñascales_MinMax,district_Las Sedas - El Olivar_MinMax,district_Latina_MinMax,district_Los Llanos - Valle Pardo_MinMax,district_Moncloa_MinMax,district_Montserrat - Parque Empresarial_MinMax,district_Noroeste_MinMax,district_Nuevo Aranjuez-Ciudad de las Artes_MinMax,district_Parla Este_MinMax,district_Parque Europa - Los Pitufos_MinMax,district_Parque Inlasa_MinMax,district_Parque Roma - Coronas_MinMax,district_Parque de la Coruña - Las Suertes_MinMax,district_Pintores-Ferial_MinMax,district_Pol. Industrial sur_MinMax,district_Pryconsa - Poligono Europa_MinMax,district_Puente de Vallecas_MinMax,district_Retiro_MinMax,district_Reyes_MinMax,district_Reyes Católicos_MinMax,district_Rivas Futura_MinMax,district_San Blas_MinMax,district_San Crispín - La Estación Consorcio_MinMax,district_San Isidro_MinMax,district_San Isidro - Los Almendros_MinMax,district_San José - Buenos Aires_MinMax,district_Sudeste Industrial_MinMax,district_Suroeste_MinMax,district_Tetuán_MinMax,district_Universidad_MinMax,district_Usera_MinMax,district_Val_MinMax,district_Valdemorillo pueblo_MinMax,district_Valdepelayo - Montepinos - Arroyo Culebro_MinMax,district_Valderas - Los Castillos_MinMax,district_Valleaguado - La Cañada_MinMax,district_Vega de la Moraleja_MinMax,district_Vicálvaro_MinMax,district_Villa de Vallecas_MinMax,district_Villalba Estación_MinMax,district_Villaverde_MinMax,district_Villayuventus-Renfe_MinMax,district_Zona Estación_MinMax,district_Zona Estación- Centro_MinMax,district_Zona Industrial_MinMax,district_Zona Pueblo_MinMax,district_desconocido_MinMax
0,750.0,0.353982,0.000000,1.000000,0.121497,1.0,0.662146,1.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,750.0,0.442478,0.590423,0.924629,0.282949,1.0,0.168104,0.379266,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_null_encoded.to_csv("../../datos/datos4/datos_nulos_encoded_minmax.csv")